<a href="https://colab.research.google.com/github/dqniellew1/DLPT/blob/master/dlpt_tensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
a = [1.0, 2.0, 1.0]

In [2]:
a[0]

1.0

In [0]:
import torch

Constructing first tensor

In [4]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [5]:
a[1]

tensor(1.)

In [0]:
a[2] = 2.0

In [8]:
a

tensor([1., 1., 2.])

Sample: Coordinate points

In [0]:
points = torch.zeros(6)
points[0] = 4.0
points[1] = 1.0
points[2] = 5.0
points[3] = 3.0
points[4] = 2.0
points[5] = 1.0


In [10]:
points

tensor([4., 1., 5., 3., 2., 1.])

Pass a python list into a tensor

In [11]:
points = torch.tensor([4.0, 1.0, 5.0, 3.0 ,2.0, 1.0])
points

tensor([4., 1., 5., 3., 2., 1.])

Get coordinates of the first point

In [14]:
float(points[0]), float(points[1])

(4.0, 1.0)

2D tensor

In [15]:
points = torch.tensor([[4.0, 2.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 2.],
        [5., 3.],
        [2., 1.]])

In [16]:
points.shape

torch.Size([3, 2])

Initialize tensor with zeros and ones

In [17]:
points = torch.zeros(3,2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [18]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [19]:
points[0,1]

tensor(1.)

In [20]:
points[0]

tensor([4., 1.])

Indexing tensors

In [0]:
some_list = list(range(6))

In [30]:
some_list[:] # returns all elements of the list
some_list[1:4] # returns elements at position 1 to 3 excluding 4
some_list[1:] # returns all elements after position 1
some_list[:4] # returns all elements up to 4 
some_list[:-1] # returns all elements from the start to one before the last element
some_list[1:4:2] # returns elements from 1 to 3 with steps of 2

[1, 3]

In [42]:
points[1:] # 1st row onwards
points[1:, :] # 1st row onwards and all columns
points[1:, 0] # 1st row onwards and 0th column
points[None] # Add dimension of size one, just like unsqueeze

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

Imagine we have a 3D tensor, we want to convert it to grayscale.

In [0]:
img_t = torch.randn(3, 5, 5) # shape [channels, rows, columns]
weights = torch.tensor([0.2126, 0.7152, 0.0722]) # typical weights for colors to derive a single brightnes value

Pretend we have an additional dimension; of batch size

In [0]:
batch_t = torch.randn(2, 3, 5, 5) # shape [batch, channels, rows, columns]

Sometimes RGB channels are in dimension 0 or 1. We can generalize by counting from the end: That is, they are always in dimension -3. 

In [55]:
img_gray_naive = img_t.mean(-3) # get mean of channels
batch_gray_naive = batch_t.mean(-3) # get mean of channels
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [81]:
# Can only multiply things that are of the same shape
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1) # Add 2 dimensions
img_weights = (img_t * unsqueezed_weights) # Multiply by same D
batch_weights = (batch_t * unsqueezed_weights) # broadcasting appends leading dimensions of size one automatically.
img_gray_weights = img_weights.sum(-3)
batch_gray_weighted = batch_weights.sum(-3)
batch_weights.shape, batch_t.shape, unsqueezed_weights.shape

(torch.Size([2, 3, 5, 5]), torch.Size([2, 3, 5, 5]), torch.Size([3, 1, 1]))

Above operations gets messy quickly, therefore the PyTorch function `einsum` makes it easier.

In [84]:
img_gray_weighted_fancy = torch.einsum('...chw,c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...chw,c->...hw', batch_t, weights)
batch_gray_weighted_fancy.shape

torch.Size([2, 5, 5])

There is quite some bookkeeping invloved when doing tensor operations, therefore practitioners suggest giving names to the dimensions instead.

Enter named tensors

In [88]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels']);
weights_named

tensor([0.2126, 0.7152, 0.0722], names=('channels',))

When we already have tensors and only want to add names, we can call the method `refine_names`. the ellipsis `...` allows you to leave out any number of dimensions. With the `rename` sibling you can also overwrite or drop (by passing in `None`) existing names.

In [101]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print(f'img named: {img_named.shape}, {img_named.names}')
print(f'batch named: {batch_named.shape}, {batch_named.names}')

img named: torch.Size([3, 5, 5]), ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]), (None, 'channels', 'rows', 'columns')


In addition to the dimension checks, eg same size, or broadcasted, PyTorch will check names for us. It does not automatically align dimensions, so we need to do this explicitly. `align_as` returns a tensor with missing dimensions added and existing ones permuted to the right order.

In [103]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

Functions accepting dimension arguments eg: `sum`, `mean` also take named dimensions

In [105]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

Combining dimensions with different names give an **error**!

In [110]:
gray_named= (img_named[..., :3] * weights_named).sum('channels')


RuntimeError: ignored

If we want to use tensors outside functions to operate on named tensors, we have to drop names by renaming them with `None`.

In [111]:
gray_plain = gray_named.rename(None)
gray_plain.shape, gray_plain.names

(torch.Size([5, 5]), (None, None))